In [1]:
# main.py
import torch
import os
import numpy as np
import matplotlib.pyplot as plt # Import matplotlib for plt.show()
import traceback # For detailed error reporting

# --- Assuming these modules exist in your project structure ---
import config
from environment import Environment # Assumes Environment class exists
from agent import PGAgent
# Assumes these functions now handle SUMO start AND close internally
from simulation import run_simulation_reinforce
from testing import run_simulation_test
from plotting import (generate_plots, # For single run diagnostics (optional)
                      generate_aggregate_plot, # Now makes train/test plots
                      plot_all_individual_runs, # Now makes train/test plots
                      plot_standard_error, # Now makes train/test plots
                      plot_run_average_distributions, # Now makes train/test plots
                      # NEW IMPORTS for PV plots
                      generate_aggregate_plot_PV,
                      plot_all_individual_runs_PV,
                      plot_standard_error_PV,
                      plot_run_average_distributions_PV
                     )
# -------------------------------------------------------------

def main():
    # Use settings from config file
    num_runs_config = config.num_runs
    train_episodes_config = config.train_episodes
    test_episodes_config = config.test_episodes
    episode_length_config = config.episode_length
    sumo_config_file = config.sumoConfig
    sumo_binary_path = config.sumoBinary
    learning_rate = config.alpha
    discount_factor = config.gamma
    # Adjust state space size based on your specific implementation
    num_states_config = 6**4 # MODIFY THIS if your state representation is different

    # Define the directory to save plots
    plot_output_dir = r"/home/narayan/Desktop/My Sumo/Stage 13(Homogeneous Network)/Results/Run_1"
    print(f"Plots will be saved in the directory: '{plot_output_dir}'")
    os.makedirs(plot_output_dir, exist_ok=True) # Create directory if it doesn't exist

    print(f"Starting experiment: {num_runs_config} runs...")
    print(f"Config: Train Eps={train_episodes_config}, Test Eps={test_episodes_config}, Ep Length={episode_length_config}")
    print(f"Agent: LR={learning_rate}, Gamma={discount_factor}, States={num_states_config}")
    print("-" * 60)

    # Lists to collect results from successful runs
    all_runs_train_avg_waits = []
    all_runs_train_avg_waits_PV = []
    all_runs_test_avg_waits = []
    all_runs_test_avg_waits_PV = []
    all_runs_train_vehicles = [] # If needed for generate_plots
    all_runs_test_vehicles = []  # If needed for generate_plots
    successful_runs = 0

    for run in range(num_runs_config):
        print(f"\n--- Starting Run {run + 1} / {num_runs_config} ---")
        run_successful = False
        env = None # Initialize env for potential use (e.g., config reading)
        try:
            # 1. Create Environment object (contains config paths)
            #    We are NOT calling env.start_simulation() here anymore.
            env = Environment(sumo_config_file, sumo_binary_path)
            print("Environment object created.")

            # 2. Try to get number of actions BEFORE simulation starts.
            #    *** This assumes env.get_num_actions() can work without a live TraCI connection ***
            #    If this line fails, you MUST modify environment.py or simulation functions
            #    to get num_actions reliably.
            num_actions = env.get_num_actions()
            print(f"Number of actions detected: {num_actions}")
            if num_actions <= 0:
                raise ValueError("Environment reported zero or negative actions. Check env.get_num_actions().")

            # 3. Initialize policy-gradient agent
            agent = PGAgent(
                num_states=num_states_config,
                num_actions=num_actions,
                lr=learning_rate,
                gamma=discount_factor
            )
            print("Agent initialized.")

            # 4. --- Training Phase ---
            #    run_simulation_reinforce is now expected to handle
            #    starting AND closing the SUMO simulation internally.
            print(f"--- Calling Training Simulation (Run {run + 1}) ---")
            _, train_avg_waits, train_avg_waits_PVeh, train_vehicles = run_simulation_reinforce(
                environment=env, # Pass the env object (might use config paths)
                agent=agent,
                episodes=train_episodes_config,
                episode_length=episode_length_config
            )
            print(f"Training simulation function returned for Run {run + 1}.")

            # 5. --- Testing Phase ---
            #    run_simulation_test is also expected to handle
            #    starting AND closing the SUMO simulation internally.
            print(f"--- Calling Testing Simulation (Run {run + 1}) ---")
            _, test_avg_waits, test_avg_waits_PVeh, test_vehicles = run_simulation_test(
                environment=env,
                agent=agent,
                episodes=test_episodes_config,
                episode_length=episode_length_config
            )
            print(f"Testing simulation function returned for Run {run + 1}.")

            # Store results only if both phases completed without exceptions in this block
            all_runs_train_avg_waits.append(train_avg_waits)
            all_runs_train_avg_waits_PV.append(train_avg_waits_PVeh)
            all_runs_test_avg_waits.append(test_avg_waits)
            all_runs_test_avg_waits_PV.append(test_avg_waits_PVeh)
            all_runs_train_vehicles.append(train_vehicles) # Optional
            all_runs_test_vehicles.append(test_vehicles)  # Optional
            successful_runs += 1
            run_successful = True

            # --- Optional: Generate diagnostic plots for this specific run ---
            # generate_plots(train_avg_waits, test_avg_waits, train_vehicles, test_vehicles,
            #                train_episodes_config, test_episodes_config, run_index=run+1)
            # print(f"-> Diagnostic plots for Run {run+1} generated (not saved automatically).")
            # generate_plots(train_avg_waits_PVeh, test_avg_waits_PVeh, train_vehicles, test_vehicles, # Example for PV
            #                train_episodes_config, test_episodes_config, run_index=run+1)
            # print(f"-> Diagnostic PV plots for Run {run+1} generated (not saved automatically).")


        except Exception as e:
            print(f"\n!!!!!! ERROR occurred in Run {run + 1} !!!!!!")
            print(f"Error Type: {type(e).__name__}")
            print(f"Error Details: {e}")
            print("Traceback:")
            traceback.print_exc()
            # Give specific hint if get_num_actions failed pre-simulation
            if isinstance(e, ValueError) and "zero or negative actions" in str(e):
                print("\nHint: Failed to get number of actions BEFORE simulation started.")
                print("Ensure `env.get_num_actions()` can work without a live SUMO connection,")
                print("or modify simulation functions to return `num_actions`.")
            elif "TraCIException" in str(type(e)):
                print("\nHint: TraCIException occurred. Check the simulation functions")
                print("(`run_simulation_reinforce`/`_test`) to ensure they manage")
                print("`traci.start()` and `traci.close()` correctly and without conflict.")

            print(f"!!!!!! Skipping rest of Run {run + 1} !!!!!!\n")

        finally:
            # No need to call env.close_simulation() here anymore,
            # as simulation functions are expected to handle it.
            print(f"--- Run {run + 1} {'Finished (Successfully)' if run_successful else 'Finished (Failed)'} ---")
            # Adding a small safeguard message
            if not run_successful:
                print("Note: Ensure simulation functions properly close SUMO on failure too.")


    # --- Experiment Finished - Generate Summary Plots ---
    print("\n" + "=" * 60)
    print(f"--- All {num_runs_config} Runs Attempted ---")
    print(f"--- {successful_runs} Runs Completed Successfully ---")
    print("--- Generating Final Summary Plots ---")
    print("=" * 60)

    if successful_runs > 0:
        # Use the actual number of successful runs for plotting
        actual_num_runs = successful_runs

        # 1. Generate the plot with all individual runs distinctly colored (Standard)
        plot_all_individual_runs(
            all_train_waits=all_runs_train_avg_waits,
            all_test_waits=all_runs_test_avg_waits,
            num_runs=actual_num_runs,
            train_episodes=train_episodes_config,
            test_episodes=test_episodes_config,
            output_dir=plot_output_dir # Pass the output directory
        )

        # 1.5. Generate the plot with all individual runs distinctly colored (PV Metric)
        plot_all_individual_runs_PV(
            all_train_waits_PV=all_runs_train_avg_waits_PV,
            all_test_waits_PV=all_runs_test_avg_waits_PV,
            num_runs=actual_num_runs,
            train_episodes=train_episodes_config,
            test_episodes=test_episodes_config,
            output_dir=plot_output_dir # Pass the output directory
        )


        # 2. Generate the aggregate plot (mean +/- stderr overlaying individuals) (Standard)
        generate_aggregate_plot(
            all_train_waits=all_runs_train_avg_waits,
            all_test_waits=all_runs_test_avg_waits,
            num_runs=actual_num_runs,
            train_episodes=train_episodes_config,
            test_episodes=test_episodes_config,
            output_dir=plot_output_dir # Pass the output directory
        )

        # 2.5. Generate the aggregate plot (mean +/- stderr overlaying individuals) (PV Metric)
        generate_aggregate_plot_PV(
            all_train_waits_PV=all_runs_train_avg_waits_PV,
            all_test_waits_PV=all_runs_test_avg_waits_PV,
            num_runs=actual_num_runs,
            train_episodes=train_episodes_config,
            test_episodes=test_episodes_config,
            output_dir=plot_output_dir # Pass the output directory
        )

        # 3. Generate the standard error plot (Standard)
        plot_standard_error(
            all_train_waits=all_runs_train_avg_waits,
            all_test_waits=all_runs_test_avg_waits,
            num_runs=actual_num_runs,
            train_episodes=train_episodes_config,
            test_episodes=test_episodes_config,
            output_dir=plot_output_dir # Pass the output directory
        )

        # 3.5. Generate the standard error plot (PV Metric)
        plot_standard_error_PV(
            all_train_waits_PV=all_runs_train_avg_waits_PV,
            all_test_waits_PV=all_runs_test_avg_waits_PV,
            num_runs=actual_num_runs,
            train_episodes=train_episodes_config,
            test_episodes=test_episodes_config,
            output_dir=plot_output_dir # Pass the output directory
        )


        # 4. Generate the run average distribution plots (Standard)
        plot_run_average_distributions(
            all_train_waits=all_runs_train_avg_waits,
            all_test_waits=all_runs_test_avg_waits,
            num_runs=actual_num_runs,
            output_dir=plot_output_dir # Pass the output directory
        )

        # 4.5. Generate the run average distribution plots (PV Metric)
        plot_run_average_distributions_PV(
            all_train_waits_PV=all_runs_train_avg_waits_PV,
            all_test_waits_PV=all_runs_test_avg_waits_PV,
            num_runs=actual_num_runs,
            output_dir=plot_output_dir # Pass the output directory
        )

        # --- Show all generated plots ---
        # Note: Since we are now saving figures and closing them within the plotting functions,
        # plt.show() will only display figures that were NOT saved and closed.
        # If you want to display the saved plots, you would need to open them
        # from the saved files or modify the plotting functions to *not* close them.
        # The current setup saves all plots to files.
        print(f"\nAll plots saved to the '{plot_output_dir}' directory.")
        # plt.show() # Uncomment this if you want to also display plots interactively

    else:
        print("\nNo runs completed successfully. Skipping final plot generation.")

    print("\n--- Experiment Script Finished ---")


if __name__ == "__main__":
    try:
        main()
    except Exception as main_e:
        print("\n!!!!!! CRITICAL ERROR in main execution !!!!!!")
        print(f"Error Type: {type(main_e).__name__}")
        print(f"Error Details: {main_e}")
        print("Traceback:")
        traceback.print_exc()

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Plots will be saved in the directory: '/home/narayan/Desktop/My Sumo/Stage 13(Homogeneous Network)/Results/Run_1'
Starting experiment: 10 runs...
Config: Train Eps=300, Test Eps=100, Ep Length=10800
Agent: LR=0.04, Gamma=0.88, States=1296
------------------------------------------------------------

--- Starting Run 1 / 10 ---
Environment object created.
Number of actions detected: 35
Agent initialized.
--- Calling Training Simulation (Run 1) ---

Episode Results
Format: Episode | Total Wait Time | Vehicles Waiting Time Per Cycle | Wait Time per Vehicle | Total Unique States
--------------------------------------------------------------------------------------------------------------------
 Retrying in 1 seconds
Step #10800.00 (1ms ~= 1000.00*RT, ~486000.00UPS, TraCI: 100ms, vehicles TOT 3515 ACT 486 tep #4500.00 (1ms ~= 1000.00*RT, ~417000.00UPS, TraCI: 1ms, vehicles TOT 1624 ACT 417 BUFep #9000.00 (1ms ~= 1000.00*RT, ~482000.00UPS, TraCI: 0ms, vehicles TOT 2995 ACT 482 BUF
#  1 |    